In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator
from IPython.display import clear_output
from datetime import datetime

start=(datetime.now())

payload=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
payload=payload[payload!='BRK.B']
payload=payload[payload!='BF.B']
#print(payload)


class algo:
    start="2010-01-01"
    end=['2022-01-01','2021-01-01','2020-01-01','2019-01-01']
    interval='1mo'

    dff=pd.DataFrame()
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        df=(yf.download(self.stock,algo.start,algo.end[0],interval=algo.interval).dropna())
        
        #df['log_current']=np.log(df.Close/df.Open)
        df['log_current']=((df.Close-df.Open)/df.Open)
        df['stock']=self.stock
        df['Year'] = df.index.strftime('%Y-%m-%d').str[:4]

        #output = stock data from start to end
        clear_output(wait=True)
        return(df)
    
    
    def stockloop():
        l1=0
        l2=50
        dff=algo.dff
        boom=pd.DataFrame()
        for i in range (l1,l2):
            try:
                rawdata=algo(payload[i]).getdata()
                dff=pd.concat([dff,rawdata])
            except Exception as e:
                print(e)
                pass
        dff.set_index([dff.stock,dff.index],inplace=True)
        return(dff)

dff=algo.stockloop()
uniq=(dff.index.unique(level=0))

finish = datetime.now()
print('Duration: {}'.format(finish - start))


Duration: 0:00:10.703189


In [2]:
start=(datetime.now())

class algo2:
    dfmass=pd.DataFrame()
    thr1=1
    thr2=99   
    end=['2022-01-01','2021-01-01','2020-01-01','2019-01-01','2018-01-01','2017-01-01','2016-01-01','2015-01-01','2014-01-01']
    def __init__(self,stock):
        self.stock=stock

    def indicators(self):
        df=self.stock
        #df['log_past']=DailyLogReturnIndicator(close=df.Close.shift(1)).daily_log_return()
        df['condition']=((df.Close-df.Close.shift(1))/df.Close.shift(1)).shift(1)
        #df['condition']=RSIIndicator(close=df.Close.shift(1)).rsi()
        #df['condition']=ROCIndicator(close=df.Close.shift(1)).roc()
        df=df.dropna()

        #output = stock data with indicators
        return(df)

    def percentiles(self):
        df=self.stock
        dic1={}
        dic2={}
        for t in range (1,len(algo2.end)):
            try:
                time=algo2.end[t]
                #print(time)
                df=(df[df.index<time])
                #print(df)
                dic1[time]=np.percentile((df.condition),algo2.thr1)
                dic2[time]=np.percentile((df.condition),algo2.thr2)
                #print(dic1,dic2)
            except:
                pass
        #output = 2 dictionaries: year - percentile of preveious year (1,99)
        return(dic1,dic2)

    def trade(self,perc1,perc2):
        df=self.stock
        dff=pd.DataFrame()
        for t in range (1,5):
            origin=algo2.end[t]
            end=algo.end[t-1]
            #print(orgin,end)
            #select origin<stock data<end
            dft=(df[(df.index>=origin) & (df.index <end)])
            
            #apply trade conditions based on thresholds
            dft['trade_pass']=np.select([(dft.condition<perc1[origin]),(dft.condition>perc2[origin])],[1,-1],default=0)
            dft['Bottom_Percentile']=perc1[origin]
            dft['Top_Percentile']=perc2[origin]
            dff=pd.concat([dff,dft])
        return(dff)
    def stockloop():
        boom=pd.DataFrame()
        for i in uniq:
            #try:
            rawdata=(dff.loc[i])
            ta=algo2(rawdata).indicators()
            percentiles=algo2(ta).percentiles()
            trades=algo2(ta).trade(algo2(ta).percentiles()[0],algo2(ta).percentiles()[1])
            boom=pd.concat([boom,trades])
            
           # except Exception as e:
             #   print(e)
               # pass
        return(boom)


boom=algo2.stockloop()
#print(boom)
boom.to_csv('boomtest.csv')

finish = datetime.now()
print('Duration: {}'.format(finish - start))


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Duration: 0:00:01.965863


In [5]:
def yearloop(boom):

    years=np.array(['2021','2020','2019','2018','2017','2016'])
    temp=np.array([])
    for i in range(0,4):
        y=years[i]
        #print(y)
        boomy=boom[boom.Year==y]
        #print(boom)
        long=boomy[boomy['trade_pass']==1]
        short=boomy[boomy['trade_pass']==-1]        
        short['log_current']=-short['log_current']
        group=([boomy,long,short])
        #print(long)
        long.to_csv('looong.csv')
        for i in range(1,2): 
            trade_count=group[i].shape[0]
            win_rate=(((group[i][group[i]['log_current']>0]).shape[0]))/trade_count
            avg_per_trade=np.average(group[i].log_current)/trade_count
            minimum=np.min(group[i].log_current)
            temp1=np.array([y,trade_count,win_rate,avg_per_trade,minimum])
            temp=np.append(temp,temp1)
    return(temp)

daf=(yearloop(boom).reshape(4,5))
dff=pd.DataFrame(daf)
print(dff.round(1))
dff.to_csv('donut.csv')

      0   1                   2                      3                     4
0  2021   4                0.75   0.012780766979327075  -0.01725597102404776
1  2020  42  0.6666666666666666  0.0021718309040763143  -0.44011799080301595
2  2019  19  0.9473684210526315   0.007045431645953147  -0.07387548056089237
3  2018  19  0.5789473684210527  0.0005469650360078758   -0.2820955863823001


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
